In [2]:
import csv
import jieba
import keras
jieba.enable_parallel(32)
import pandas as pd
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import *
from keras.optimizers import *
from keras.layers.core import *
from keras.layers import Input,merge, TimeDistributed
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras import backend as K

np.random.seed(1337)

Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/da/miniconda3/envs/idp3/lib/python3.5/site-packages/jieba/__init__.py", line 152, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmp36au5a9f' -> '/tmp/jieba.cache'
Loading model cost 3.190 seconds.
Prefix dict has been built succesfully.


In [4]:
# 参数设置
maxlen = 81
epochs = 5
batch = 32
emb = 300

In [22]:
import tensorflow as tf

In [8]:
def get_aspect(X):
    ans = X[:, 0, :]
    return ans

def get_context(X):
    ans = X[:, 1:, :]
    return ans

def get_R(X):
    Y, alpha = X[0], X[1]
#     ans = K.dot(Y, alpha)
    ans = K.dot(Y, K.transpose(alpha))
    return ans
def naive_get_shape():
    return (300, 1)
def expend_shape(x):
    return K.expand_dims(x, axis=-1)

# alpha = Lambda(expend_shape, output_shape=(maxlen-1,1))(alpha)
# r_ = Lambda(get_R, output_shape=naive_get_shape())([w_context_trans, alpha])

In [9]:
print('Build model...')
main_input = Input(shape=(maxlen,), dtype='int32', name='main_input')
# x = Embedding(output_dim=emb, input_dim=len(dict)+1, input_length=maxlen, name='x')(main_input)
x = Embedding(output_dim=emb, input_dim=20000, input_length=maxlen, name='x')(main_input)
drop_out = Dropout(0.1, name='dropout')(x)
w_aspect = Lambda(get_aspect, output_shape=(emb,), name="w_aspect")(drop_out)
w_context = Lambda(get_context, output_shape=(maxlen-1,emb), name="w_context")(drop_out)
w_aspect = Dense(emb, kernel_regularizer=l2(0.01), name="w_aspect_1")(w_aspect)
# hop 1
w_aspects = RepeatVector(maxlen-1, name="w_aspects1")(w_aspect)
merged = keras.layers.concatenate([w_context, w_aspects], name='merged1')
distributed = TimeDistributed(Dense(1, kernel_regularizer=l2(0.01), activation='tanh'), name="distributed1")(merged)
flat_alpha = Flatten(name="flat_alpha1")(distributed)
alpha = Dense(maxlen-1, activation='softmax', name="alpha1")(flat_alpha)

alpha = Lambda(expend_shape, output_shape=(maxlen-1,1))(alpha)

w_context_trans = Permute((2, 1), name="w_context_trans1")(w_context)
# r_ = keras.layers.merge([w_context_trans, alpha], output_shape=(emb, 1), name="r_1", mode=get_R)
r_ = Lambda(get_R, output_shape=(emb, 1))([w_context_trans, alpha])
r = Reshape((emb,), name="r1")(r_)
w_aspect_linear = Dense(emb, kernel_regularizer=l2(0.01), activation='linear')(w_aspect)
merged = keras.layers.add([r, w_aspect_linear])
w_aspect = Dense(emb, kernel_regularizer=l2(0.01), name="w_aspect_2")(merged)
# hop 2
w_aspects = RepeatVector(maxlen-1, name="w_aspects2")(w_aspect)
merged = keras.layers.concatenate([w_context, w_aspects], name='merged2')
distributed = TimeDistributed(Dense(1, kernel_regularizer=l2(0.01), activation='tanh'), name="distributed2")(merged)
flat_alpha = Flatten(name="flat_alpha2")(distributed)
alpha = Dense(maxlen-1, activation='softmax', name="alpha2")(flat_alpha)
w_context_trans = Permute((2, 1), name="w_context_trans2")(w_context)
# r_ = keras.layers.merge([w_context_trans, alpha], output_shape=(emb, 1), name="r_2", mode=get_R)
r_ = Lambda(get_R, output_shape=(emb, 1))([w_context_trans, alpha])
r = Reshape((emb,), name="r2")(r_)
w_aspect_linear = Dense(emb, kernel_regularizer=l2(0.01), activation='linear')(w_aspect)
merged = keras.layers.add([r, w_aspect_linear])
w_aspect = Dense(emb, kernel_regularizer=l2(0.01), name="w_aspect_3")(merged)
# hop 3
w_aspects = RepeatVector(maxlen-1, name="w_aspects3")(w_aspect)
merged = keras.layers.concatenate([w_context, w_aspects], name='merged3')
distributed = TimeDistributed(Dense(1, kernel_regularizer=l2(0.01), activation='tanh'), name="distributed3")(merged)
flat_alpha = Flatten(name="flat_alpha3")(distributed)
alpha = Dense(maxlen-1, activation='softmax', name="alpha3")(flat_alpha)
w_context_trans = Permute((2, 1), name="w_context_trans3")(w_context)
# r_ = keras.layers.merge([w_context_trans, alpha], output_shape=(emb, 1), name="r_3", mode=get_R)
r_ = Lambda(get_R, output_shape=(emb, 1))([w_context_trans, alpha])
r = Reshape((emb,), name="r3")(r_)
w_aspect_linear = Dense(emb, kernel_regularizer=l2(0.01), activation='linear')(w_aspect)
merged = keras.layers.add([r, w_aspect_linear])
h_ = Activation('tanh')(merged)
out = Dense(3, activation='softmax')(h_)
output = out
model = Model(inputs=[main_input], outputs=output)
model.summary()

Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 81)            0                                            
____________________________________________________________________________________________________
x (Embedding)                    (None, 81, 300)       6000000                                      
____________________________________________________________________________________________________
dropout (Dropout)                (None, 81, 300)       0                                            
____________________________________________________________________________________________________
w_aspect (Lambda)                (None, 300)           0                                            
____________________________________________________________________________

In [33]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: No module named 'keras.utils.visualize_util'